In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [12]:
# Function to extract Product Title
def get_title(soup):

    try:
        title = soup.find("span", attrs={"id":'productTitle'})
        
        title_value = title.text

        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-price a-text-price a-size-medium apexPriceToPay'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'a-span12'}).string.strip()

        except:
            price = ""

    return price

def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=Indian+Flag&crid=2Z4O4OM17BU8S&sprefix=indian+flag%2Caps%2C305&ref=nb_sb_noss_1"

    webpage = requests.get(URL, headers=HEADERS)

    soup = BeautifulSoup(webpage.content, "html.parser")

    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    links_list = []

    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))
   
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [11]:
amazon_df

,title,price,rating,reviews,availability
1,PlayStation®4 Console – Call of Duty® Modern W...,,4.4 out of 5 stars,70 ratings,In Stock
2,PlayStation®5 Digital Edition (slim),,4.7 out of 5 stars,"5,545 ratings",In Stock
3,Wireless Controller Dual Vibration Game Joysti...,,4.1 out of 5 stars,"1,773 ratings",In Stock
4,PlayStation 4 Slim 500GB Console [Discontinued],,4.6 out of 5 stars,"1,893 ratings",In Stock
5,PlayStation 4 Slim 1TB Console - Fortnite Bund...,,4.5 out of 5 stars,6 ratings,Not Available
6,OIVO PS4 Stand Cooling Fan Station for Playsta...,,4.5 out of 5 stars,"45,496 ratings",In Stock
7,Wireless Controller for PS4 with 2 Thumb Grips...,,4.2 out of 5 stars,"2,203 ratings",In Stock
8,"Charger Dock Station for PS4, 1.8 Hrs Fast Cha...",,4.6 out of 5 stars,"14,511 ratings",In Stock
9,DualShock 4 Wireless Controller for PlayStatio...,,3.9 out of 5 stars,"1,766 ratings",In Stock
10,PlayStation 4 Slim 1TB Console - Marvel's Spid...,,4.7 out of 5 stars,"2,817 ratings",Only 1 left in stock - order soon.
